In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np 
import pandas as pd 
import os, cv2, random, time, shutil, csv
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
np.random.seed(42)
%matplotlib inline 

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

In [2]:
#Data Paths
train_dir = '../input/dog-breed-identification/train'
test_dir = '../input/dog-breed-identification/test'
#Count/Print train and test samples.
data_size = len(os.listdir(train_dir))
test_size = len(os.listdir(test_dir))
print('Data samples size: ', data_size)
print('Test samples size: ', test_size)

Data samples size:  10222
Test samples size:  10357


In [3]:
labels_dataframe = pd.read_csv('../input/dog-breed-identification/labels.csv')


sample_df = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

labels_dataframe.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
dog_breeds = sorted(list(set(labels_dataframe['breed'])))
n_classes = len(dog_breeds)
print(n_classes)
dog_breeds[:10]

120


['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle']

In [5]:
class_to_num = dict(zip(dog_breeds, range(n_classes)))

In [6]:
def images_to_array(data_dir, labels_dataframe, img_size = (299,299,3)):
    
    images_names = labels_dataframe['id']
    images_labels = labels_dataframe['breed']
    data_size = len(images_names)
    #initailize output arrays.
    X = np.zeros([data_size, img_size[0], img_size[1], img_size[2]], dtype=np.uint8)
    y = np.zeros([data_size,1], dtype=np.uint8)
    #read data and lables.
    for i in tqdm(range(data_size)):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
        image_breed = images_labels[i]
        y[i] = class_to_num[image_breed]
    
    #One hot encoder
    y = to_categorical(y)
    #shuffle    
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y

In [7]:
img_size = (299,299,3)
X, y = images_to_array(train_dir, labels_dataframe, img_size)

100%|██████████| 10222/10222 [00:47<00:00, 217.26it/s]


Ouptut Data Size:  (10222, 299, 299, 3)
Ouptut Label Size:  (10222, 120)


In [8]:
def get_features(model_name, data_preprocessor, input_size, data):
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=64, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [9]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

160/160 [==============================] - 52s 323ms/step
Feature maps shape:  (10222, 1536)


In [10]:
final_features = np.concatenate([inc_resnet_features,], axis=-1)
print('Final feature maps shape', final_features.shape)

Final feature maps shape (10222, 1536)


In [12]:
dnn = keras.models.Sequential([
    InputLayer(final_features.shape[1:]),
    Dropout(0.7),
    Dense(n_classes, activation='softmax')
])

dnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Train simple DNN on extracted features.
h = dnn.fit(final_features, y,
            batch_size=128,
            epochs=60,
            validation_split=0.1)

Epoch 1/60
72/72 [==============================] - 0s 6ms/step - loss: 2.3803 - accuracy: 0.5948 - val_loss: 0.5892 - val_accuracy: 0.8974
Epoch 2/60
72/72 [==============================] - 0s 3ms/step - loss: 0.5081 - accuracy: 0.8882 - val_loss: 0.3461 - val_accuracy: 0.9091
Epoch 3/60
72/72 [==============================] - 0s 3ms/step - loss: 0.3614 - accuracy: 0.9048 - val_loss: 0.3015 - val_accuracy: 0.9140
Epoch 4/60
72/72 [==============================] - 0s 3ms/step - loss: 0.3131 - accuracy: 0.9097 - val_loss: 0.2886 - val_accuracy: 0.9101
Epoch 5/60
72/72 [==============================] - 0s 3ms/step - loss: 0.2887 - accuracy: 0.9172 - val_loss: 0.2906 - val_accuracy: 0.9101
Epoch 6/60
72/72 [==============================] - 0s 3ms/step - loss: 0.2666 - accuracy: 0.9154 - val_loss: 0.2873 - val_accuracy: 0.9169
Epoch 7/60
72/72 [==============================] - 0s 3ms/step - loss: 0.2552 - accuracy: 0.9198 - val_loss: 0.2888 - val_accuracy: 0.9120
Epoch 8/60
72/72 [==